In [23]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import glob

In [24]:
ecad_custom = requests.get('https://www.ecad.eu/dailydata/customquery.php?optionSelected=element&processtext1=Your+query+is+being+processed.+Please+wait...&blendingselect=yes&countryselect=GERMANY%7Cde&stationselect=All+stations%7C**&elementselect=Mean+temperature%7Ctg&processtext2=Your+query+is+being+processed.+Please+wait...')

In [25]:
# requests.text will return the html file of the website as a string

In [26]:
# print(ecad_custom.text) # hmtl string

In [27]:
soup = BeautifulSoup(ecad_custom.text, 'lxml')
# print(soup)

In [28]:
# get all berlin station names
s = soup.find('select', attrs={"name":"stationselect"})
berlin_stations = []
for option in s.find_all('option'):
    station_name = option.get('value')
    if 'BERLIN-' in station_name:
        berlin_stations.append(station_name)

print(berlin_stations)
    

['BERLIN-ADLERSHOF[de]|11736', 'BERLIN-BUCH[de]|4529', 'BERLIN-DAHLEM (LFAG)[de]|4533', 'BERLIN-DAHLEM[de]|41', 'BERLIN-FRIEDRICHSHAGEN[de]|11737', 'BERLIN-FROHNAU[de]|11738', 'BERLIN-INVALIDENSTRASSE[de]|11739', 'BERLIN-KANISWALL[de]|4546', 'BERLIN-LICHTENRADE[de]|4556', 'BERLIN-LICHTERFELDE (SUD)[de]|4559', 'BERLIN-MARZAHN[de]|4561', 'BERLIN-MITTE[de]|4563', 'BERLIN-OSTKREUZ[de]|11740', 'BERLIN-RUDOW[de]|4566', 'BERLIN-SCHONEFELD[de]|4570', 'BERLIN-SPANDAU[de]|4575', 'BERLIN-TEGELER FLIESSTAL[de]|4581', 'BERLIN-TEGEL[de]|4005', 'BERLIN-TEMPELHOF[de]|2759', 'BERLIN-TREPTOW[de]|4586', 'BERLIN-ZEHLENDORF[de]|4588']


In [29]:
# remove the first few lines from the txt files

In [30]:
for file in os.listdir('data/ECA_blended_custom/'):
    #print(file)
    if file.startswith('TG_STAID'):
        current_file = file
        print(current_file)
        
        with open('data/ECA_blended_custom/' + current_file, "r") as f:
            content = f.readlines()
            content = content[18:]
            #print(content)
            #print(type(content)) # is a list
            string ='\n'.join(content)
            new_f = open(f"data/berlin_temp_clean/cleaned_{current_file}", "w") 
            new_f.write(string)

TG_STAID002759.txt
TG_STAID011737.txt
TG_STAID011736.txt
TG_STAID004533.txt
TG_STAID000041.txt


In [31]:
import re

In [32]:
# impott all temperature files and concatenate to one big dataframe
path = 'data/berlin_temp_clean/' # use your path
all_files = glob.glob(path + "clean*.txt")



print(all_files[0])
pattern = 'STAID\w+'
station = re.findall('STAID\w+', path)
print(station)


data/berlin_temp_clean/cleaned_TG_STAID011736.txt
[]


In [89]:
# impott all temperature files and concatenate to one big dataframe
path = 'data/berlin_temp_clean/' 
all_files = glob.glob(path + "clean*.txt")

temp_file_list = []

for filename in all_files:
    station = re.findall('(0+)(\d*)', filename)[0][1]
    station_name = ""
    for berlin_station in berlin_stations:
        berlin_station_name = re.match('\w{6}\-\w{2,}[^\[]', berlin_station)
        if station in berlin_station:
            station_name = berlin_station_name.group(0)

    single_df = pd.read_csv(filename, index_col=None, header=0).assign(Station=station, Station_name=station_name)
    
    temp_file_list.append(single_df)

df = pd.concat(temp_file_list, axis=0, ignore_index=True)

In [90]:
df.head()

,SOUID,DATE,TG,Q_TG,Station,Station_name
0,180888,19480701,126,0,11736,BERLIN-ADLERSHOF
1,180888,19480702,160,0,11736,BERLIN-ADLERSHOF
2,180888,19480703,126,0,11736,BERLIN-ADLERSHOF
3,180888,19480704,160,0,11736,BERLIN-ADLERSHOF
4,180888,19480705,174,0,11736,BERLIN-ADLERSHOF


In [35]:
df['Station'].value_counts

<bound method IndexOpsMixin.value_counts of 0         'STAID011736'
1         'STAID011736'
2         'STAID011736'
3         'STAID011736'
4         'STAID011736'
              ...      
211077    'STAID000041'
211078    'STAID000041'
211079    'STAID000041'
211080    'STAID000041'
211081    'STAID000041'
Name: Station, Length: 211082, dtype: object>

In [82]:
df['Station_name'] = ""
df.head()

,SOUID,DATE,TG,Q_TG,Station,Station_name
0,180888,19480701,126,0,11736,
1,180888,19480702,160,0,11736,
2,180888,19480703,126,0,11736,
3,180888,19480704,160,0,11736,
4,180888,19480705,174,0,11736,


In [36]:
df.shape

(211082, 5)

In [77]:
berlin_stations

['BERLIN-ADLERSHOF[de]|11736',
 'BERLIN-BUCH[de]|4529',
 'BERLIN-DAHLEM (LFAG)[de]|4533',
 'BERLIN-DAHLEM[de]|41',
 'BERLIN-FRIEDRICHSHAGEN[de]|11737',
 'BERLIN-FROHNAU[de]|11738',
 'BERLIN-INVALIDENSTRASSE[de]|11739',
 'BERLIN-KANISWALL[de]|4546',
 'BERLIN-LICHTENRADE[de]|4556',
 'BERLIN-LICHTERFELDE (SUD)[de]|4559',
 'BERLIN-MARZAHN[de]|4561',
 'BERLIN-MITTE[de]|4563',
 'BERLIN-OSTKREUZ[de]|11740',
 'BERLIN-RUDOW[de]|4566',
 'BERLIN-SCHONEFELD[de]|4570',
 'BERLIN-SPANDAU[de]|4575',
 'BERLIN-TEGELER FLIESSTAL[de]|4581',
 'BERLIN-TEGEL[de]|4005',
 'BERLIN-TEMPELHOF[de]|2759',
 'BERLIN-TREPTOW[de]|4586',
 'BERLIN-ZEHLENDORF[de]|4588']

,SOUID,DATE,TG,Q_TG,Station,Station_name
0,180888,19480701,126,0,11736,BERLIN-ADLERSHOF
1,180888,19480702,160,0,11736,BERLIN-ADLERSHOF
2,180888,19480703,126,0,11736,BERLIN-ADLERSHOF
3,180888,19480704,160,0,11736,BERLIN-ADLERSHOF
4,180888,19480705,174,0,11736,BERLIN-ADLERSHOF
